In [ ]:
# Sprouting angiogenesis – finite‑difference (Colab‑ready, fibronectin kymographs)
# -------------------------------------------------------------------------------------------
# • Tracks tip‑cell density n, new sprouts ∆n, and fibronectin F deposition over time.
# • Fibronectin is produced by tip cells (α1) and degrades (γ1). We visualize its
#   spatiotemporal field alongside n and ∆n with inline animations.
# -------------------------------------------------------------------------------------------
import numpy as np, math, scipy.sparse as sp, scipy.sparse.linalg as spla
import matplotlib.pyplot as plt, matplotlib.animation as animation
from IPython.display import HTML
from matplotlib import rc; rc('animation', html='jshtml')

# # ------------------------------ PARAMETERS ------------------------------
# T, num_steps = 1, 1200          # total time & steps (dt = T/num_steps)
# Nx = Ny = 161                     # grid resolution

# dx   = 1.0/(Nx-1)
# Dt   = T/num_steps                # time increment
# Dn   = 0.0005                      # diffusion (thinner, thread‑like sprouts)                       # diffusion of tip cells
# chi  = 0.04                        # stronger chemotaxis for alignment                        # chemotaxis strength
# rho  = 0.08                        # haptotaxis to fibronectin
# Dm   = 1                      # lower random motility for thread coherence                       # random motility amplitude

# hex_pitch, line_th = 0.15, 0.005  # hex lattice parameters
# hex_high, hex_low = 0.8, 0.2      # chemo values on ridges vs interior
# c_bnd = 0.2                       # vessel‑wall chemo level

# num_tips = 4                      # number of initial sprouts
# sig_x, sig_y = 0.005, 0.003       # cluster spread
# seed = 123                        # RNG seed

# # fibronectin kinetics
# #   α1: rate at which tip cells produce fibronectin each time step
# #   γ1: rate at which fibronectin degrades each time step
# #   Adjust α1 ↑ to increase deposition, γ1 ↑ to accelerate clearance
# γ1 = 0.4  # degradation rate
# α1 = 0.05   # production rate by tip cells

# # output cadence
# out_every, frame_every = 10, 10

# ------------------------------ PARAMETERS ------------------------------
T, num_steps = 1.5, 1200          # total time & steps (dt = T/num_steps)
Nx = Ny = 401                   # grid resolution

dx   = 1.0/(Nx-1)
Dt   = T/num_steps              # time increment
Dn   = 1e-1                    # approx. nondimensionalized diffusion from Dn = 10^-10 cm²/s
chi  = 0.3                      # chemotaxis strength from χ₀ = 2600 cm²/s/M
rho  = 0*0.08                      # haptotaxis (estimated to show matrix-guided migration)
Dm   = 0*0.1                        # random motility amplitude

hex_pitch, line_th = 0.15, 0.005  # hex lattice parameters
hex_high, hex_low = 0.8,1      # chemo values on ridges vs interior
c_bnd = 0.2                       # vessel‑wall chemo level

num_tips = 4                      # number of initial sprouts
sig_x, sig_y = 0.005, 0.003       # cluster spread
seed = 123                        # RNG seed

# Fibronectin kinetics
γ1 = 0.05   # degradation rate (slower to preserve gradients)
α1 = 0.1    # production rate by tip cells (higher to match buildup)

# output cadence
out_every, frame_every = 10, 10

# -------------------------- INITIAL GRIDS --------------------------
rng = np.random.default_rng(seed)
X, Y = np.meshgrid(np.linspace(0,1,Nx), np.linspace(0,1,Ny), indexing='ij')
vec   = lambda A: A.ravel(order='F')
shape = lambda v: v.reshape((Nx,Ny), order='F')
wall_idx = np.arange(Nx)

# create random-offset hex ridges (±60° only)
s3 = math.sqrt(3)
off_x = rng.uniform(0, hex_pitch)
off_y = rng.uniform(0, hex_pitch*s3/2)
Xh = (X+off_x)%1; Yh = (Y+off_y)%1
plus  = (((Yh+s3*Xh)%(s3*hex_pitch))<line_th)
minus = (((Yh-s3*Xh)%(s3*hex_pitch))<line_th)
ridge = plus | minus

# chemo field
c = np.where(ridge, hex_high, hex_low).astype(float)
c[:,0] = c_bnd

# initial tip cells
n0 = np.zeros_like(c)
for x0 in rng.uniform(0.1,0.9,num_tips):
    n0 += np.exp(-((X-x0)**2)/sig_x**2) * np.exp(-(Y/sig_y)**2)
n0 = np.clip(n0, 0, 1)

# initial fibronectin
F0 = np.zeros_like(c)

# --------------------- FINITE‑DIFFERENCE SETUP ---------------------
Ntot = Nx*Ny
main = -4*np.ones(Ntot)
east = np.ones(Ntot-1); east[np.arange(1,Nx)*Nx-1]=0
west = np.ones(Ntot-1); west[np.arange(Nx)]=0
north= np.ones(Ntot-Nx); south=np.ones(Ntot-Nx)
L = sp.diags([main,east,west,north,south],[0,1,-1,Nx,-Nx],shape=(Ntot,Ntot)).tolil()
for i in range(Nx):
    L[i,i]+=1; L[(Ny-1)*Nx+i,(Ny-1)*Nx+i]+=1
L = (L/dx**2).tocsr(); I = sp.identity(Ntot,format='csr')
solve_n = spla.factorized(I - Dt*Dn*L)

# state vectors
C = vec(c)
Ntip = vec(n0)
wall_supply = Ntip.copy()
F = vec(F0)
frames_n, frames_dn, frames_F = [], [], []
print("⚙️ Simulation starting…")
for step in range(num_steps):
    # reshape fields for finite difference
    n_grid = shape(Ntip); f_grid = shape(F); c_grid = shape(C)
    # compute gradients of c and F
    dcx = np.zeros_like(c_grid); dcy = np.zeros_like(c_grid)
    dcx[1:-1,:] = (c_grid[2:,:] - c_grid[:-2,:])/(2*dx)
    dcy[:,1:-1] = (c_grid[:,2:] - c_grid[:,:-2])/(2*dx)
    dfx = np.zeros_like(f_grid); dfy = np.zeros_like(f_grid)
    dfx[1:-1,:] = (f_grid[2:,:] - f_grid[:-2,:])/(2*dx)
    dfy[:,1:-1] = (f_grid[:,2:] - f_grid[:,:-2])/(2*dx)
    # fluxes
    Jx = chi * n_grid * dcx + rho * n_grid * dfx
    Jy = chi * n_grid * dcy + rho * n_grid * dfy
    divJ = np.zeros_like(n_grid)
    divJ[1:-1,:] = (Jx[2:,:] - Jx[:-2,:])/(2*dx)
    divJ[:,1:-1] += (Jy[:,2:] - Jy[:,:-2])/(2*dx)
    # update tip-cell density
    Ntip = solve_n(Ntip + Dt*vec(divJ))
    # random motility
    Ntip += np.sqrt(2*Dm*Dt)*rng.standard_normal(Ntot)
    Ntip = np.clip(Ntip, 0, None)
    # wall replenishment
    Ntip[wall_idx] = wall_supply[wall_idx]
    # update fibronectin
    F = F + Dt * (α1 * Ntip - γ1 * F)
    F = np.clip(F, 0, None)
    # record frames
    if step % frame_every == 0:
        frames_n.append(shape(Ntip))
        frames_dn.append(shape(Ntip - wall_supply))
        frames_F.append(shape(F))
    if step % out_every == 0:
        print(f" step {step}/{num_steps}  t={(step+1)*Dt:.3f}")
print("✅ Simulation complete")

# ---------------------------- PLOTS & ANIMATIONS ----------------------------
# ------------------------ STATIC SNAPSHOTS ------------------------
fig,axs=plt.subplots(2,3,figsize=(18,8))
# Chemo
axs[0,0].imshow(c.T,origin='lower',extent=[0,1,0,1]); axs[0,0].set_title('Chemo (c)')
# Tip cells
axs[0,1].imshow(frames_n[-1].T,origin='lower',extent=[0,1,0,1]); axs[0,1].set_title('Tip cells n – final')
# New sprouts
axs[0,2].imshow(frames_dn[-1].T,origin='lower',extent=[0,1,0,1]); axs[0,2].set_title('New sprouts ∆n final')
# Fibronectin
axs[1,0].imshow(frames_F[-1].T,origin='lower',extent=[0,1,0,1],cmap='inferno'); axs[1,0].set_title('Fibronectin F – final')
# Overlay ∆n on chemo
overlay = np.ma.masked_where(frames_dn[-1] < 1e-3, frames_dn[-1])
axs[1,1].imshow(c.T,origin='lower',extent=[0,1,0,1],cmap='Greys',alpha=0.3)
axs[1,1].imshow(overlay.T,origin='lower',extent=[0,1,0,1],cmap='spring',alpha=0.8)
axs[1,1].set_title('Overlay ∆n on c')
# Binary vessel network (thresholded ∆n)
binary = frames_dn[-1] > 0.05
axs[1,2].imshow(binary.T,origin='lower',extent=[0,1,0,1],cmap='gray')
axs[1,2].set_title('Binary vessels (∆n > 0.05)')
plt.tight_layout(); plt.show()

# -------------------------- INLINE ANIMATIONS --------------------------
print("📽️ Rendering animations…")

def build(frames, title, cmap=None):
    fig,ax = plt.subplots(); im = ax.imshow(frames[0].T,origin='lower',extent=[0,1,0,1], cmap=cmap)
    ax.set_title(title)
    def upd(i):
        im.set_data(frames[i].T)
        ax.set_xlabel(f't = {(i*frame_every)*Dt:.3f}')
        return [im]
    ani = animation.FuncAnimation(fig, upd, frames=len(frames), interval=120, blit=True)
    plt.close(fig)
    return ani

ani_n  = build(frames_n,  'Tip‑cell density n')
ani_dn = build(frames_dn, 'New sprouts ∆n')
ani_F  = build(frames_F,  'Fibronectin F', cmap='inferno')

HTML(ani_n.to_jshtml())

#HTML(ani_dn.to_jshtml())
#HTML(ani_F.to_jshtml())
